### Import Modules

In [26]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [32]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Specify fit parameters

In [40]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 0, # max 10
        'exclude' : [], # put LECs here
        'include_log' : True, # alphaS
        'include_log2' : True
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'ma', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    'chained' : False,
    
    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_prior' : False,
    'save_results' : False,
}

### Do fit

In [41]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(p_dict['fit_type'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print bootstrapper
    
if p_dict['save_prior']:
    print 'Saving prior...'
    data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.901999950409
Compiling results...


Fit type: ma (F^2 = FKFpi, bsN = 1)

Fitting to nnlo 
 with volume corrections O(0) 
 chained: False 
Fitted/[FLAG] values at physical point (including SU(2) isospin corrections):

F_K / F_pi = 1.1867(47) [1.1932(19)]   (delta_su2 = -0.00213(21))

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
      L_4 0             0.0040 (41)     A_loga 0              -0.3 (4.2)
          1             0.0000 (50)            1               0.0 (5.0)
      L_5 0             0.0008 (11)        A_a 0              -1.1 (2.9)
          1             0.0000 (50)            1               0.0 (5.0)
      A_p 0               0.3 (1.1)        A_k 0               1.3 (2.0)
          1               0.0 (5.0)            1               0.0 (5.0)
---
boot0 fit results:
Least Square Fit:
  chi2/dof [dof] = 2.1

In [35]:
print bootstrapper.fits[0].pmean['L_5']

0.0019417849006856534


In [11]:
print np.__version__

1.16.5


In [29]:
print bootstrapper.fits[0].pmean['L_5']

0.000434627622523


In [8]:
myfit = bootstrapper.fits[0]

In [9]:
myfit.fitter

'scipy_least_squares'

In [42]:
translator_dict = {
    'xpt' : 'xpt-ratio', 
    'xpt-taylor' : 'xpt',
    'ma' : 'ma-ratio', 
    'ma-taylor' : 'ma',
}

def make_table(bootstrapper):
    name = translator_dict[bootstrapper.fit_type]
    if bootstrapper.order['vol'] > 0:
        name = name + '_FV'

    if bootstrapper.order['include_log']:
        name = name + '_alphaS'

    if bootstrapper.order['include_log2']:
        name = name + '_logSq'
        
    fit = bootstrapper.extrapolate_to_phys_point(include_su2_isospin_corrrection=False)
    print name, fit
    print bootstrapper.fits[0]
    
make_table(bootstrapper)

ma-ratio_alphaS_logSq 1.1880(48)
Least Square Fit:
  chi2/dof [dof] = 2.1 [16]    Q = 0.0075    logGBF = 56.711

Parameters:
          mpi 0        0.097917 (63)         [   0.097896 (63) ]  
              1         0.14074 (12)         [    0.14074 (12) ]  
              2         0.15781 (20)         [    0.15783 (20) ]  
              3         0.18114 (15)         [    0.18114 (15) ]  
              4         0.08131 (17)         [    0.08134 (17) ]  
              5         0.13423 (17)         [    0.13427 (17) ]  
              6         0.13403 (15)         [    0.13402 (15) ]  
              7         0.13575 (31)         [    0.13556 (32) ]  
              8         0.18869 (17)         [    0.18868 (17) ]  
              9         0.21400 (20)         [    0.21397 (20) ]  
             10         0.24344 (16)         [    0.24346 (16) ]  
             11         0.10282 (11)         [    0.10282 (12) ]  
             12         0.16531 (19)         [    0.16531 (19) ]  
    

In [354]:
    def fk_fpi_fit_fcn(self, fit_data=None, fit_parameters=None, fit_type=None):
        if fit_data is None:
            fit_data = self.get_phys_point_data()
        if fit_parameters is None:
            fit_parameters = self.get_fit_parameters().copy()
        if fit_type is None:
            fit_type = 'xpt-taylor'

        for key in fit_data.keys():
            fit_parameters[key] = fit_data[key]
            
        temp_prior = self._randomize_prior(self.prior, j)
        
        temp_fitter = fit.fitter(fit_data=fit_data, prior=temp_prior, 
                                 order=self.order, fit_type=fit_type, chain_fits=self.chain_fits)
        fitfcn = temp_fitter._make_models()[0].fitfcn
        
        print fitfcn(p=fit_parameters)

        #return temp_fit.fcn(p=fit_parameters)[model_name]
    
    
    fk_fpi_fit_fcn(bootstrapper)

1.1853(37)


In [266]:
fit = bootstrapper.fits[0]

In [270]:
bootstrapper.get_fit_parameters()

{'A_a': 0.4(2.8),
 'A_k': -0.9(2.4),
 'A_loga': 1.1(4.2),
 'A_p': 1.19(44),
 'L_5': 0.00043(57)}

In [280]:
params = [bootstrapper.get_fit_parameters()[key] for key in ['L_5', 'A_k', 'A_p']]
gv.evalcorr(params)

array([[ 1.        , -0.99758431,  0.88976061],
       [-0.99758431,  1.        , -0.90501669],
       [ 0.88976061, -0.90501669,  1.        ]])

In [283]:
gv.gvar(130.2/np.sqrt(2), 1.7/np.sqrt(2))

92.1(1.2)

In [284]:
gv.gvar(155.6/np.sqrt(2), 0.4/np.sqrt(2))

110.03(28)

In [ ]:
phys_point['Fpi'] = gv.gvar(130.2/np.sqrt(2), 1.7/np.sqrt(2)) #PDG fpi+ eq(16)
phys_point['FK']  = gv.gvar(155.6/np.sqrt(2), 0.4/np.sqrt(2)) #PDG fK++ eq(16)

In [355]:
def test(self):
    for abbr in self.abbrs:
        print abbr, self.fit_data[abbr]['FK/Fpi'], self.extrapolate_to_ensemble(abbr)

test(bootstrapper)

a09m220 1.1439(18) 1.1456(14)
a09m310 1.1108(16) 1.11142(71)
a09m350 1.0997(10) 1.09829(71)
a09m400 1.08107(94) 1.08069(73)
a12m130 1.1646(26) 1.1622(16)
a12m220 1.1314(23) 1.1334(11)
a12m220L 1.1279(29) 1.1328(11)
a12m220S 1.1389(27) 1.1359(12)
a12m310 1.1042(21) 1.10516(82)
a12m350 1.0888(27) 1.0926(10)
a12m400 1.0745(17) 1.07627(89)
a15m135XL 1.1584(28) 1.1573(21)
a15m220 1.1266(15) 1.1269(12)
a15m310 1.1016(13) 1.10020(88)
a15m350 1.08868(87) 1.08843(78)
a15m400 1.07151(89) 1.07206(80)


### Or make all fits

In [44]:
p_dict = {
    'order' : {
        'vol' : 5, # max 10
        'exclude' : [],
        'include_log2' : True
    }, 
    'bs_N' : 1,  # if 0, use full list
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    'chained' : True,

    'use_prior' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400', u'a15m130', u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],

    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,

    'save_prior' : False,
    'save_results' : True,
}

In [45]:
for fit_type in ['xpt', 'xpt-taylor', 'ma', 'ma-taylor']:#, 'ma-old']:
    for order_fit in ['nlo', 'nnlo']:#, 'nnnlo']:
        
        print fit_type, order_fit
        
        # Specify fit parameters
        p_dict['order']['fit'] = order_fit
        p_dict['fit_type'] = fit_type
        
        # Do fit
        t0 = time.time()

        # Load data
        data_loader = dl.data_loader()
        fit_data = data_loader.get_fit_data()

        # Get prior
        prior = None
        if p_dict['use_prior']:
            prior = data_loader.get_prior(p_dict['fit_type'])

        # Make bootstrapper
        bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                                       fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

        if p_dict['make_plots']:
            data_loader.save_plots(
                bootstrapper.make_plots(
                    show_error_ellipses=p_dict['show_error_ellipses'], 
                    show_bootstrap_histograms=p_dict['show_bs_histograms']),
                output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
            )
        else:
            print bootstrapper

        if p_dict['save_prior']:
            print 'Saving prior...'
            data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])

        if p_dict['save_results']:
            data_loader.save_fit_info(bootstrapper.get_fit_info())


        t1 = time.time()
        
        #print test(bootstrapper)

        print "\nTotal time (s): ", t1 - t0, "\n"

xpt nlo
Making fits...
100% complete Time (s):  0.463999986649
Compiling results...


../fitter\special_functions.py:10: RuntimeWarning: invalid value encountered in double_scalars
  ysdev = np.abs(ss.kvp(n, gv.mean(g), 1)) * gv.sdev(g)


Done.
Saving...
Done.

Total time (s):  11.7779998779 

xpt nnlo
Making fits...
100% complete Time (s):  0.979000091553
Compiling results...
Done.
Saving...
Done.

Total time (s):  12.881000042 

xpt-taylor nlo
Making fits...
100% complete Time (s):  0.468999862671
Compiling results...
Done.
Saving...
Done.

Total time (s):  8.63800001144 

xpt-taylor nnlo
Making fits...
100% complete Time (s):  0.984000205994
Compiling results...
Done.
Saving...
Done.

Total time (s):  9.89199995995 

ma nlo
Making fits...
100% complete Time (s):  0.991000175476
Compiling results...
Done.
Saving...
Done.

Total time (s):  54.1529998779 

ma nnlo
Making fits...
100% complete Time (s):  2.49399995804
Compiling results...
Done.
Saving...
Done.

Total time (s):  56.1860001087 

ma-taylor nlo
Making fits...
100% complete Time (s):  1.11199998856
Compiling results...
Done.
Saving...
Done.

Total time (s):  49.6940000057 

ma-taylor nnlo
Making fits...
100% complete Time (s):  2.46700000763
Compiling results

In [38]:
#bootstrapper.fits[0].show_plots(view='std')

### Compare with Andre

In [10]:
p_dict = {
    'order' : {
        'fit' : 'nnlo'
    }, 
    'bs_N' : 1,  # if 0, use full list
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    'chained' : False,

    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
}

In [11]:
results_dict = {}

for fit_type in ['xpt', 'ma', 'xpt-taylor', 'ma-taylor']:#, 'ma-old']:
    results_dict[fit_type] = {}
    for include_log2 in [True, False]:
        results_dict[fit_type][include_log2] = {}
        for include_log in [True, False]:
            results_dict[fit_type][include_log2][include_log] = {}
            for vol in [0, 10]:
                results_dict[fit_type][include_log2][include_log][vol] = {}
                #print fit_type, order_fit

                # Specify fit parameters
                p_dict['order']['include_log'] = include_log
                p_dict['order']['include_log2'] = include_log2
                p_dict['order']['vol'] = vol

                p_dict['fit_type'] = fit_type

                # Do fit

                # Load data
                data_loader = dl.data_loader()
                fit_data = data_loader.get_fit_data()

                # Get prior
                prior = None

                # Make bootstrapper
                bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])


                results_dict[fit_type][include_log2][include_log][vol]['fit'] = bootstrapper.extrapolate_to_phys_point(include_su2_isospin_corrrection=False)
                #results_dict[fit_type][include_log2][include_log][vol]['L5'] = bootstrapper.get_fit_parameters('L_5')
                results_dict[fit_type][include_log2][include_log][vol]['info'] = bootstrapper.get_fit_info()
                #results_dict[fit_type][include_log2][include_log][vol]['fit'] = 
                print 

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.541000127792
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.585999965668
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.532999992371
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.574000120163
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.47000002861
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.544000148773
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.65499997139
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.555000066757
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  1.25100016594
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  

In [18]:
translator_dict = {
    'xpt' : 'xpt-ratio', 
    'xpt-taylor' : 'xpt',
    'ma' : 'ma-ratio', 
    'ma-taylor' : 'ma',
}

#print "name   FK/Fpi   L5   k   p   s"

k = 30 - len("name")
output = "name" +k*' '

for param in ['FK/Fpi', 'L5', 'k', 'p', 's']:
    k = 14 - len(param)
    output += k*" " + str(param)

print output
                        
                        
print "---"

#for fit_type in results_dict.keys():
    #for include_log2 in results_dict[fit_type].keys():
for include_log2 in results_dict['ma'].keys():
    for include_log in results_dict[fit_type][include_log2].keys():
        for vol in results_dict[fit_type][include_log2][include_log].keys():
            for fit_type in ['xpt-taylor', 'ma-taylor']:
                
                name = translator_dict[fit_type]
                if vol > 0:
                    name = name + '_FV'
                    
                if include_log:
                    name = name + '_alphaS'
                
                if include_log2:
                    name = name + '_logSq'
                    
                    
                fit = results_dict[fit_type][include_log2][include_log][vol]['fit']
                L5 = results_dict[fit_type][include_log2][include_log][vol]['info']['L_5']
                Ak = results_dict[fit_type][include_log2][include_log][vol]['info']['A_k']
                Ap = results_dict[fit_type][include_log2][include_log][vol]['info']['A_p']
                Aa = results_dict[fit_type][include_log2][include_log][vol]['info']['A_a']
                
                if ((vol==0), include_log, include_log2) in [(1, 1, 1), (1, 1, 0), (1, 0, 0), (0, 0, 0)]:
                    if len(name) > 0:
                        k = 30 - len(name)
                        output = name +k*' '
                        
                        for param in [fit, L5, Ak, Ap, Aa]:
                            k = 14 - len(str(param))
                            output += k*" " + str(param)
                            
                        print output
                
                    


name                                  FK/Fpi            L5             k             p             s
---
xpt                               1.1811(25)  -0.00035(49)      2.3(2.1)      1.59(39)     -3.32(36)
ma                                1.1812(23)   0.00041(57)     -0.8(2.4)      1.18(44)      1.17(40)
xpt_FV                            1.1821(27)  -0.00059(49)      3.3(2.1)      1.36(39)     -3.85(36)
ma_FV                             1.1811(23)   0.00034(57)     -0.5(2.4)      1.11(44)      0.96(40)
xpt_alphaS                        1.1806(38)  -0.00037(50)      2.3(2.1)      1.59(39)     -2.9(2.8)
ma_alphaS                         1.1820(36)   0.00043(57)     -0.9(2.4)      1.19(44)      0.4(2.8)
xpt_alphaS_logSq                  1.1832(36)  -0.00032(50)      0.1(2.2)      2.24(40)     -2.9(2.8)
ma_alphaS_logSq                   1.1855(36)   0.00006(58)     -1.3(2.4)      1.53(44)      0.4(2.8)


In [23]:
print np.__version__
print matplotlib.__version__
print gv.__version__
print lsqfit.__version__

1.13.3
2.1.0
9.2
11.2


In [24]:
print python__version__

NameError: name 'python__version__' is not defined